In [ ]:
import pandas as pd
import json
import numpy as np
from dcf.dcf_widget import DCFWidget

In [ ]:
df = pd.read_csv('./examples/data/2014-01-citibike-tripdata.csv')
df2 = pd.read_csv('./examples/data/yellow_tripdata_2021-02.csv')

In [ ]:
w = DCFWidget(df=df2)
w

In [ ]:
transformed_df = w.dcf_transform(w.commands, w.df)
w.transformed_df = json.loads(transformed_df.to_json(orient='table', indent=2))

In [ ]:
w.commands

In [ ]:
w.generated_py_code

In [ ]:
w.generated_py_code = w.dcf_to_py_core(w.commands[1:])
w.generated_py_code

In [ ]:
from dcf.all_transforms import Transform
from dcf.lispy import s

In [ ]:
@w.add_command
class GroupBy2(Transform):
    command_default = [s("groupby2"), s('df'), 'col', {}]
    command_pattern = [[3, 'colMap', 'colEnum', ['null', 'sum', 'mean', 'median', 'count']]]
    @staticmethod 
    def transform(df, col, col_spec):
        grps = df.groupby(col)
        df_contents = {}
        for k, v in col_spec.items():
            if v == "sum":
                df_contents[k] = grps[k].apply(lambda x: x.sum())
            elif v == "mean":
                df_contents[k] = grps[k].apply(lambda x: x.mean())
            elif v == "median":
                df_contents[k] = grps[k].apply(lambda x: x.median())
            elif v == "count":
                df_contents[k] = grps[k].apply(lambda x: x.count())
        return pd.DataFrame(df_contents)

    #test_df = group_df
    #test_sequence = [s("groupby2"), s('df'), 'c', dict(a='sum', b='mean')]
    test_output = pd.DataFrame(
        {'a':[100, 110], 'b':[2.5, 5.5]},
        index=['q','w'])

    @staticmethod 
    def transform_to_py(df, col, col_spec):
        commands = [
            "    grps = df.groupby('%s')" % col,
            "    df_contents = {}"
        ]
        for k, v in col_spec.items():
            if v == "sum":
                commands.append("    paddydf_contents['%s'] = grps['%s'].apply(lambda x: x.sum())" % (k, k))
            elif v == "mean":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.mean())" % (k, k))
            elif v == "median":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.median())" % (k, k))
            elif v == "count":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.count())" % (k, k))
        #print("commands", commands)
        commands.append("    df = pd.DataFrame(df_contents)")
        return "\n".join(commands)
